# Fine-tuning CodeLlama 7B for Code Review using Unsloth on Google Colab

This notebook guides you through fine-tuning the `codellama/CodeLlama-7b-hf` model to act as a code reviewing tool. We will use the Unsloth library for efficient, memory-optimized training (specifically 4-bit QLoRA) suitable for a free Google Colab environment (like the T4 GPU).

**Dataset:** We will use the `HuggingFaceH4/Code-Feedback` dataset, which contains dialogues involving code snippets and feedback, making it suitable for training a code review assistant.

**Goal:** To create a model that can take a piece of code as input and provide constructive review comments or suggestions.

## 1. Setup Environment

First, we need to install the necessary libraries. Unsloth handles the installation of optimized versions of `transformers`, `peft`, `accelerate`, and `bitsandbytes`.

In [ ]:
# Install Unsloth and other required libraries
!pip install "unsloth[colab-new]>=2024.5" -q
!pip install "transformers>=4.38.0" -q
!pip install "datasets[vision]>=2.16.0" -q
!pip install "accelerate>=0.28.0" -q
!pip install "trl>=0.8.6" -q
!pip install "peft>=0.10.0" -q
!pip install "bitsandbytes>=0.43.0" -q

### GPU Check

Let's verify that we have a suitable GPU available. Unsloth is optimized for NVIDIA GPUs, and free Colab typically provides a T4.

In [ ]:
# Check GPU status
!nvidia-smi

### Import Libraries

Now, import the necessary components.

In [ ]:
import torch
from datasets import load_dataset
from transformers import TrainingArguments, AutoTokenizer
from trl import SFTTrainer
from peft import LoraConfig
from unsloth import FastLanguageModel

## 2. Load Model and Tokenizer

We'll load the `codellama/CodeLlama-7b-hf` model using Unsloth's `FastLanguageModel`. This automatically applies optimizations like 4-bit quantization (QLoRA) to make it fit within Colab's memory limits.

In [ ]:
max_seq_length = 2048 # Choose based on GPU memory and typical code review context length
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere
load_in_4bit = True # Use 4-bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "codellama/CodeLlama-7b-hf",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # Optional: use if accessing gated models like Llama 3
)

### Configure PEFT (LoRA)

We configure Parameter-Efficient Fine-Tuning (PEFT) using LoRA (Low-Rank Adaptation). Unsloth helps automatically find the optimal modules to target.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # Modules to apply LoRA to
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True, # Significantly saves memory
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## 2.5. Test Model Before Fine-tuning

Let's test the model's performance before fine-tuning to see how it responds to code review requests. This will serve as a baseline for comparison after training.

In [ ]:
# Test the model before fine-tuning
print("Testing model BEFORE fine-tuning...")

# Define the same prompt template we'll use for training
test_prompt_template = """Below is a conversation between a user asking for code review and an AI assistant providing feedback.
### Conversation:
{}
### Feedback:
"""

# Example code snippet for review (same as we'll use later for comparison)
test_code = """
**User:** Can you review this Python function?

```python
def add_numbers(a, b):
  # This function adds two numbers
  result = a+b
  return result
```
"""

# Format the input
test_input = test_prompt_template.format(test_code.strip())
test_inputs = tokenizer([test_input], return_tensors="pt").to("cuda")

# Generate response before fine-tuning
print("Generating response before fine-tuning...")
with torch.no_grad():
    test_outputs = model.generate(
        **test_inputs, 
        max_new_tokens=256, 
        use_cache=True,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the output
test_decoded = tokenizer.batch_decode(test_outputs)[0]

# Extract only the generated feedback part
feedback_start = test_decoded.find("### Feedback:") + len("### Feedback:")
generated_before = test_decoded[feedback_start:].strip()

# Clean up the output
if generated_before.endswith(tokenizer.eos_token):
    generated_before = generated_before[:-len(tokenizer.eos_token)].strip()

print("--- Input Code ---")
print(test_code)
print("--- Generated Feedback (BEFORE fine-tuning) ---")
print(generated_before)
print("\n" + "="*50)
print("Now proceeding with fine-tuning...")

## 3. Load and Prepare Dataset

We load the `HuggingFaceH4/Code-Feedback` dataset and format it into a structure suitable for supervised fine-tuning (SFT). We'll create a simple prompt template that presents the code and asks for a review.

In [ ]:
# Define a prompt template for code review
# The dataset has 'instruction', 'output', and 'messages' fields.
# We'll use the 'messages' field which contains a list of turns.
# We format it into a single string resembling a conversation.

prompt_template = """Below is a conversation between a user asking for code review and an AI assistant providing feedback.
### Conversation:
{}
### Feedback:
{}
"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    formatted_texts = []
    for i in range(len(examples['messages'])):
        conversation = ""
        messages = examples['messages'][i]
        # Find the last assistant message as the target output
        if messages[-1]['role'] == 'assistant':
            output = messages[-1]['content']
            # Concatenate previous messages for context
            for msg in messages[:-1]:
                conversation += f"**{msg['role'].capitalize()}:** {msg['content']}\n"
            conversation = conversation.strip()
            # Apply the template
            text = prompt_template.format(conversation, output) + EOS_TOKEN
            formatted_texts.append(text)
        # Handle cases where the last message isn't from the assistant (optional, could skip)
        # else: 
        #    pass 
            
    return { "text" : formatted_texts }


# Load the dataset
dataset = load_dataset("HuggingFaceH4/Code-Feedback", split = "train_sft") # Using the SFT split

# Apply the formatting function
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Optional: Shuffle and select a subset for faster training if needed
# dataset = dataset.shuffle(seed=42).select(range(1000)) 

print(f"Dataset prepared. Example entry:{dataset[0]['text']}")

## 4. Configure Training

Set up the training arguments using `transformers.TrainingArguments` and initialize the `SFTTrainer` from the `trl` library. We configure parameters suitable for a free Colab instance.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # Column containing the formatted text
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processes for dataset preparation
    packing = False, # Can make training faster, but requires more memory
    args = TrainingArguments(
        per_device_train_batch_size = 2, # Adjust based on GPU memory
        gradient_accumulation_steps = 4, # Effective batch size = batch_size * accumulation_steps
        warmup_steps = 5,
        # max_steps = 60, # Set a fixed number of steps for faster training (adjust as needed)
        num_train_epochs = 1, # Or train for a number of epochs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(), # Use fp16 if bf16 not supported (T4)
        bf16 = torch.cuda.is_bf16_supported(), # Use bf16 if supported (Ampere)
        logging_steps = 1,
        optim = "adamw_8bit", # Use 8-bit AdamW optimizer to save memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs", # Directory to save checkpoints and logs
    ),
)

## 5. Train the Model

Start the fine-tuning process. This may take some time depending on the dataset size and `max_steps`/`num_train_epochs`.

In [ ]:
print("Starting training...")
trainer_stats = trainer.train()
print("Training finished.")
# You can view training metrics in trainer_stats

## 6. Save the Model Adapters

After training, save the learned LoRA adapters. These adapters are much smaller than the full model and contain the fine-tuned knowledge.

In [ ]:
# Save the LoRA adapters
output_dir = "codellama_code_review_lora"
model.save_pretrained(output_dir) # Saves LoRA adapters
tokenizer.save_pretrained(output_dir) # Saves tokenizer
print(f"LoRA adapters saved to {output_dir}")

# Optional: Push to Hugging Face Hub
# from huggingface_hub import login
# login() # Log in to your Hugging Face account
# model.push_to_hub("your_username/codellama_code_review_lora", token = True)
# tokenizer.push_to_hub("your_username/codellama_code_review_lora", token = True)

## 7. Inference Example

Let's see how to use the fine-tuned model for inference. We load the base model again and apply the saved LoRA adapters.

In [ ]:
from unsloth import FastLanguageModel
import torch

# Load the base model and tokenizer again (if needed, or use the 'model' object if still in memory)
# Make sure to provide the path to your saved adapters

# Check if 'model' and 'tokenizer' are still loaded, otherwise reload
if 'model' not in locals():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "codellama_code_review_lora", # Load from your saved directory
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    print("Model loaded from saved adapters.")
else:
    # If model is still in memory, ensure it's in evaluation mode
    model.eval()
    print("Using model already in memory.")

# Define the inference prompt template (should match the training format)
inference_prompt_template = """Below is a conversation between a user asking for code review and an AI assistant providing feedback.
### Conversation:
{}
### Feedback:
"""

# Example code snippet for review
code_to_review = """
**User:** Can you review this Python function?

```python
def add_numbers(a, b):
  # This function adds two numbers
  result = a+b
  return result
```
"""

# Format the input
input_text = inference_prompt_template.format(code_to_review.strip())
inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

# Generate the feedback
print("Generating feedback...")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
decoded_output = tokenizer.batch_decode(outputs)[0]

# Extract only the generated feedback part
feedback_start = decoded_output.find("### Feedback:") + len("### Feedback:")
generated_feedback = decoded_output[feedback_start:].strip()
# Remove potential EOS token if present at the end
if generated_feedback.endswith(tokenizer.eos_token):
    generated_feedback = generated_feedback[:-len(tokenizer.eos_token)].strip()

print("--- Input Code ---")
print(code_to_review)
print("--- Generated Feedback ---")
print(generated_feedback)

## 8. Conclusion

You have successfully fine-tuned CodeLlama 7B for code review tasks using Unsloth on Google Colab!

**Next Steps:**
*   **Evaluate:** Perform more rigorous evaluation on a separate test set to measure the quality of the reviews.
*   **Experiment:** Try different hyperparameters (learning rate, LoRA rank, epochs), datasets, or prompt formats.
*   **Deploy:** Merge the adapters with the base model for easier deployment or use the adapters directly with PEFT.
*   **Push to Hub:** Share your fine-tuned adapters on the Hugging Face Hub.